In [1]:
#Social Distancing Project using YOLO

In [2]:
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
import cv2
import numpy as np
import random
import os
import time

In [3]:
def main():
    root = tk.Tk()
    root.title("Social Distancing")

    # Set the window size and make it non-resizable
    root.geometry("500x500")
    root.resizable(False, False)

    # Load the background image
    bg_image = Image.open("social.jpg")
    bg_photo = ImageTk.PhotoImage(bg_image)

    # Create a label to hold the background image
    bg_label = tk.Label(root, image=bg_photo)
    bg_label.place(x=0, y=0, relwidth=1, relheight=1)

    # Create a label for the welcome message
    welcome_label = tk.Label(root, text="Welcome to Social Distance Indicator", font=("Arial", 16, "bold"))
    welcome_label.pack(pady=20)

    def run_social_distance_detector():
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4")])
        if file_path:
            
            # Load the YOLO weights and configuration
            # 'yolov3.weights' contains pre trained weights of NN
            # 'yolov3.cfg' contains network configuration
            net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
            net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
            net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
            distance_thres = 50
            
            # Open the video file for reading
            cap = cv2.VideoCapture(file_path)
            
            def dist(pt1,pt2):
                try:
                    # calculates Euclidean distance
                    return ((pt1[0]-pt2[0])**2 + (pt1[1]-pt2[1])**2)**0.5
                except:
                    return
                
            # retrieves the names of all the layers in the neural network model 'net'
            layer_names = net.getLayerNames()
            # indentifies the output layer and retrieves the names of the layers corresponding 
            # to the unconnected output layers by subtracting 1 from each index
            output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
            #print name of the output layer
            print('Output layers',output_layers)
            # Read the first frame from the video
            _,frame = cap.read()
            
            # Create a VideoWriter object to save the output video
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter('output.avi', fourcc, 30,(frame.shape[1], frame.shape[0]), True)

            ret = True
            while ret:
                # Read the next frame from the video
                ret, img = cap.read()
                if ret:
                    height, width = img.shape[:2]
                    
                    # Preprocess the image for input to the neural network
                    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
                    net.setInput(blob)
                    outs = net.forward(output_layers)
                    confidences = []
                    boxes = []
                    
                     # Process each detected object in the output
                    for out in outs:
                        # iterates over each element within out, representing a specific detection within the image
                        for detection in out:
                            # extracts the scores which represent the confidence of the model 
                            scores = detection[5:]
                            # finds the index of the highest score 
                            class_id = np.argmax(scores)
                            if class_id!=0:
                                continue
                            confidence = scores[class_id]
                                #checks if thres is above 0.3 of the selected class
                            if confidence > 0.3:
                                # creates bounding boxes
                                center_x = int(detection[0] * width)
                                center_y = int(detection[1] * height)
                                w = int(detection[2] * width)
                                h = int(detection[3] * height)
                                x = int(center_x - w / 2)
                                y = int(center_y - h / 2)
                                boxes.append([x, y, w, h])
                                confidences.append(float(confidence))
                    
                    # Apply non-maximum suppression based on the confidence score to remove redundant overlapping boxes
                    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
                    persons = []
                    person_centres = []
                    violate = set()
                    # Iterate over the remaining boxes after non-maximum suppression
                    for i in range(len(boxes)):
                        if i in indexes:
                            x,y,w,h = boxes[i]
                            persons.append(boxes[i])
                            person_centres.append([x+w//2,y+h//2])
                    # Check for social distancing violations
                    for i in range(len(persons)):
                        for j in range(i+1,len(persons)):
                            if dist(person_centres[i],person_centres[j]) <= distance_thres:
                                violate.add(tuple(persons[i]))
                                violate.add(tuple(persons[j]))
        
                    v = 0
                    for (x,y,w,h) in persons:
                        if (x,y,w,h) in violate:
                            color = (0,0,255)
                            v+=1
                        else:
                            color = (0,255,0)
                        cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
                        cv2.circle(img,(x+w//2,y+h//2),2,(0,0,255),2)
                    cv2.putText(img,'No of Violations : '+str(v),(15,frame.shape[0]-10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,126,255),2)
                    writer.write(img)
                    cv2.imshow("Result", img)
    
                if cv2.waitKey(1) == 27:
                    break
            cap.release()
            cv2.destroyAllWindows()

            

    # Create a frame to hold the welcome message and the button
    frame = tk.Frame(root)
    frame.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

    # Create a button to select and run the social distance detection on the video file
    detect_button = tk.Button(frame, text="Select and Run Social Distance Detector", command=run_social_distance_detector, padx=10, pady=5, fg="white", bg="blue", font=("Arial", 12, "bold"))
    detect_button.pack()

    root.mainloop()

In [4]:
if __name__ == "__main__":
    main()